In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

In [0]:
DATADIR = '/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/'

CATEGORIES = [ 'agricultural',  'airplane',    'baseballdiamond', 'beach',   'buildings',          'chaparral',         'denseresidential',
                'forest',        'freeway',     'golfcourse',      'harbor',  'intersection',       'mediumresidential', 'mobilehomepark',
                'overpass',      'parkinglot',  'river',           'runway',  'sparseresidential',  'storagetanks',      'tenniscourt' ]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from PIL import Image #import Python Image Library
import numpy as np
import os
import glob
import re
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [0]:
!ls

In [0]:
path = os.path.abspath('.cnn.py')

In [0]:
print(path)

In [0]:
path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file

In [0]:
print(path)

In [0]:
dirs = os.listdir(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')

In [0]:
img_rows = 256
img_cols = 256
num_class = 21
path = os.path.abspath('.cnn.py') #absolute path of program
path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file
X = []
Y = []
dirs = os.listdir(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')
dirs=dirs[:]
print(len(dirs))

In [0]:
label = 0
for i in dirs:
	n = 0
	count = 0
	for pic in glob.glob('/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/'+i+'/*.tif'):
		im = Image.open(pic)
		im = np.array(im)
		if((im.shape[0]==256) and (im.shape[1] ==256) and n<90): #get only 90 data
			r = im[:,:,0]
			g = im[:,:,1]
			b = im[:,:,2]
			X.append([r,g,b])
			Y.append([label])
			n = n+1
	print(n)
	label = label + 1

In [0]:
print(len(X))
print(len(Y))

In [0]:
import numpy as np
X_bck=X
y_bck=Y


X =  np.array(X)
Y =  np.array(Y)
X = X.reshape(X.shape[0], img_rows, img_cols, 3)
X.shape

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [0]:
print(X_train.shape)
print(X_test.shape)

In [0]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)

input_shape = (img_rows, img_cols, 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = keras.utils.to_categorical(y_train, 21)
y_test = keras.utils.to_categorical(y_test, 21)

In [0]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print("dsa")
print(y_test[0:10])
print("dsa")
print(y_train)

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
model.fit(X_train, y_train, batch_size=50, nb_epoch=100, verbose=1, validation_data=(X_test, y_test))

In [0]:
model_big = Sequential()
model_big.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model_big.add(Conv2D(32, (3, 3), activation='relu'))
model_big.add(MaxPooling2D(pool_size=(2,2), strides=2))
model_big.add(Conv2D(64, (3, 3), activation='relu'))
model_big.add(Conv2D(64, (3, 3), activation='relu'))
model_big.add(MaxPooling2D(pool_size=(2,2), strides=2))
model_big.add(Dropout(0.25))
model_big.add(Conv2D(128, (3,3), activation='relu'))
model_big.add(MaxPooling2D(pool_size=(2,2), strides=2))
model_big.add(Flatten())
model_big.add(Dense(128, activation='relu'))
model_big.add(Dropout(0.5))
model_big.add(Dense(21, activation='softmax'))
model_big.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [0]:
model_big.summary()

In [0]:
model_big.fit(X_train, y_train, batch_size=50, nb_epoch=150, verbose=1, validation_data=(X_test, y_test))

In [0]:
loss, acc = model_big.evaluate(X_test, y_test, verbose=0)
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [0]:
!pip install  split_folders tqdm

In [0]:
import split_folders
split_folders.ratio(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/', output=path+'/drive/My Drive/ucmerced/UCMerced_LandUse/output/', seed=1337, ratio=(.8, .2))


In [0]:
from keras.preprocessing.image import ImageDataGenerator
import time
IMG_SIZE =256 # Replace with the size of your images
NB_CHANNELS =3 # 3 for RGB images or 1 for grayscale images
BATCH_SIZE = 32# Typical values are 8, 16 or 32
NB_TRAIN_IMG = 1680# Replace with the total number training images
NB_VALID_IMG = 420# Replace with the total number validation images

In [0]:
model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,NB_CHANNELS),
               data_format='channels_last'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(Conv2D(32, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(Dropout(0.25))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(21, activation='softmax'))
model1.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

#Augmentation

In [0]:
train_datagen = ImageDataGenerator(
    rotation_range = 40,                  
    width_shift_range = 0.2,                  
    height_shift_range = 0.2,                  
    rescale = 1./255,                  
    shear_range = 0.2,                  
    zoom_range = 0.2,                     
    horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(
    path+'drive/My Drive/ucmerced/UCMerced_LandUse/output/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size = BATCH_SIZE)
validation_generator = validation_datagen.flow_from_directory(
    path+'drive/My Drive/ucmerced/UCMerced_LandUse/output/val',
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size = BATCH_SIZE)

In [0]:
start = time.time()
model_big.fit_generator(
    train_generator,
    steps_per_epoch=NB_TRAIN_IMG//BATCH_SIZE,
    epochs=300,
    validation_data=validation_generator,
    validation_steps=NB_VALID_IMG//BATCH_SIZE)
end = time.time()
print('Processing time:',(end - start)/60)